In [9]:
import ollama
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional

df = pd.read_csv('../dataset/data.csv')

df.head()

,OBSRVTN_NB,DATETIME_DTM,PNT_NM,QUALIFIER_TXT,PNT_ATRISKNOTES_TX,PNT_ATRISKFOLWUPNTS_TX
0,330560,3/15/2023 11:01,"Did you recognize additional slip, trip, or fa...",Awareness of environment,[NAME] was working a near by cliff that had ab...,NaN
1,164330,7/9/2019 10:00,Vehicle Operating Condition,Other - Vehicle Operating Condition,[NAME] trucks with cut out bumbers need a hitc...,NaN
2,265239,8/1/2022 10:15,Suspended Load/Overhead Work,Other - Suspended Load/Overhead Work,Employee rigged a concrete culvert to be off l...,Coached employee that once he had the culverts...
3,404438,10/18/2023 12:06,PPE - Workforce,Other - PPE Workforce,[NAME] tender was not wearing his Hardhat. I r...,NaN
4,83172,3/3/2021 11:00,Climbing - Procedures,"Was a drop zone established, and clearly marked?",A drop zone was not clearly marked by the crew...,NaN


In [10]:
df.set_index('OBSRVTN_NB', inplace=True)
df.fillna("NA", inplace=True)
df['DATETIME_DTM'] = pd.to_datetime(df['DATETIME_DTM'])
df_subsample = df[::200]
df.head()

,DATETIME_DTM,PNT_NM,QUALIFIER_TXT,PNT_ATRISKNOTES_TX,PNT_ATRISKFOLWUPNTS_TX
OBSRVTN_NB,,,,,
330560,2023-03-15 11:01:00,"Did you recognize additional slip, trip, or fa...",Awareness of environment,[NAME] was working a near by cliff that had ab...,NA
164330,2019-07-09 10:00:00,Vehicle Operating Condition,Other - Vehicle Operating Condition,[NAME] trucks with cut out bumbers need a hitc...,NA
265239,2022-08-01 10:15:00,Suspended Load/Overhead Work,Other - Suspended Load/Overhead Work,Employee rigged a concrete culvert to be off l...,Coached employee that once he had the culverts...
404438,2023-10-18 12:06:00,PPE - Workforce,Other - PPE Workforce,[NAME] tender was not wearing his Hardhat. I r...,NA
83172,2021-03-03 11:00:00,Climbing - Procedures,"Was a drop zone established, and clearly marked?",A drop zone was not clearly marked by the crew...,NA


In [11]:
SYSTEM_PROMPT_FORMAT = """
You are a superintelligent Safety Classification and Learning (SCL) Model 
with the goal classifying field notes written on site by AEP electrical engineers and other essential personnel.

Your task is to reason with the field notes and output a classification for that field note data.

For your context, for the rest of this prompt, when we refer to High Energy Situations, High Energy Incidents, Serious Injury Sustained, and Direct Control present are as follows:
Definition of High Energy Situation:
High energy refers to situations where the physical forces or hazardous conditions involved are strong enough to greatly increase the likelihood of a serious injury or fatality (SIF). These situations can involve various forms of energy, including gravity, motion, mechanical, electrical, pressure, sound, radiation, biological, chemical, or temperature.
The presence of high energy indicates that the conditions pose a significant risk to individuals, typically involving forces or exposures that exceed safe thresholds. For example, situations such as falls from elevation, suspended loads, heavy rotating equipment, explosions, electrical contact exceeding 50 volts, high temperatures above 150°F, or high pressure in excavations over 5 feet deep are typical indicators of high energy, often resulting in severe consequences.
Even when exact measurements like voltage or force are not mentioned, hazardous conditions should be evaluated to determine if the risk level suggests the presence of high energy. In such cases, a reasonable inference can be made based on the description of the incident and the potential for serious harm.
Definition of High Energy Incident:
A high energy incident occurs when a high energy situation results in an actual event, such as an accident, injury, or near-miss. This involves a release of hazardous energy—where the energy changes state or is no longer contained—and the worker comes into contact with or is in proximity to the energy source. Contact means the high energy is transmitted to the body, while proximity refers to the worker being within 6 feet of the hazard with unrestricted egress, or any distance where escape is limited.
Definition of a Serious Injury Sustained:
Serious Injury Sustained includes work related fatalities and life-threatening and life-altering injuries.
Definition of Direct Control Present:
Direct controls are: Targeted (the barrier must specifically target a high-energy source), effective: (it must effectively mitigate exposure to the high-energy source when installed, verified, and used properly (i.e., a SIF reasonable should not occur if these conditions are met)) and have human error: (it must be effective even in the presence of unintentional human error during work that is unrelated to the installation of the control)
Examples of Direct Controls: [lockout/tagout (LOTO), machine guarding, hard physical barriers, fall protection, cover-ups]
Examples that are NOT Direct Controls: [training, warning signs, rules, experience, standard non-specialized personal protective equipment (e.g., hard hats, gloves, boots)]

The field notes have the following schema:
PNT_NM: Point name, the safety criteria being assessed. Forms a primary key when combined with OBSRVN_NB (observation number)
QUALIFIER_TXT: Qualifier text, list of predetermined observations chosen by the reviewer based on the point being assessed
PNT_ATRISKNOTES_TX: Point at-risk notes text, comments left by observer regarding unsafe conditions they found
PNT_ATRISKFOLWUPNTS_TX: Point at-risk follow up notes text, recommended remediation for at-risk conditions observed. This field may be empty (Indicated by "NA"). 

You must classify this data using these classes. The format of the classes is "class": -> "description of class":
{classes}

NOTE: your output should ONLY be the class code and NOTHING else:
"""

USER_PROMPT_FORMAT = """
Below is the field notes:

{field_notes}

REMEMBER:
NOTE: your output should ONLY be the class code and NOTHING else:
"""

In [12]:
class GenerateWeakLabels:
    def __init__(
            self,
            classes: Dict[int, str],
            column_label: str,
            shot_examples: Optional[Dict[str, str]] = None,
            df: pd.DataFrame = df_subsample,
            model: str = 'llama3:latest',
        ):  
        """Generate one decision for an llm to make. """
        predictions = []

        for _, row in df.iterrows():
            prompt_field_notes = "\n".join(f"{key}: {row[key]}" for key in ['PNT_NM', 'QUALIFIER_TXT', 'PNT_ATRISKNOTES_TX', 'PNT_ATRISKFOLWUPNTS_TX'])
            print(prompt_field_notes)

            output = ollama.chat(
                model    = model, 
                messages = self._build_shot_prompts(classes, prompt_field_notes, shot_examples),
                options  = {'temperature': 0, 'num_predict': 1}
            )

            print(output['message']['content'])
            predictions.append(output['message']['content'])

        df[column_label] = predictions

    def _schedule_classification_queries(self):
        ...
        
    def _build_shot_prompts(self, classes, prompt_field_notes, shot_examples):
        prompt_classes = "\n".join(f"{k}: -> {v}" for k, v in classes.items())

        messages = [{'role': 'system', 'content': SYSTEM_PROMPT_FORMAT.format(classes=prompt_classes)}]

        if shot_examples is None:
            messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=prompt_field_notes,)})
            #self._pretty_print_messages(messages)
            return messages

        for example in shot_examples:
            messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=example)})
            messages.append({'role': 'assistant', 'content': str(shot_examples[example])})

        messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=prompt_field_notes)})
        #self._pretty_print_messages(messages)
        return messages
    
    def _pretty_print_messages(self, messages):
        for message in messages:
            role = message['role'].capitalize()
            content = message['content']
            print(f"{role}: {content}\n")

In [13]:
classes = {
    0: 'Def: High Energy Serious Injury Sustained (HSIF). This means that there was a High Energy Incident AND there was a Serious Injury Sustained.',
    1: 'Def: Low Energy Serious Injury Sustained (LSIF). This means that there was NOT a High Energy Present AND there was a Serious Injury Sustained.',
    2: 'Def: Potential Serious Injury Sustained (PSIF). This means that there was a High Energy Incident AND there was NOT a Serious Injury Sustained.',
    3: 'Def: None of the above criteria defs (HSIF, LSIF, PSIF) are met.',
}

multishot = {
    """
    PNT_NM: Workplace Conditions Addressed
    QUALIFIER_TXT: Voltage being worked discussed
    PNT_ATRISKNOTES_TX: A crew was working near a sedimentation pond on a rainy day. The boom of the trac-hoe was within 3
    feet of a live 12kV line running across the road. No contact was made because a worker intervened and
    communicated with the operator.
    PNT_ATRISKFOLWUPNTS_TX: NA
    """: "3",
    """
    PNT_NM: Climbing - Procedures
    QUALIFIER_TXT: Was a drop zone established, and clearly marked?
    PNT_ATRISKNOTES_TX: Workers were hoisting beams and steel onto a scaffold. A certified mechanic operated an air hoist to lift the beam. As the lift was performed, the rigging was caught under an adjacent beam. Under the increasing tension, the cable snapped and struck a second employee in the leg, fully fracturing his femur. An investigation indicated that the rigging was not properly inspected before the lift.
    PNT_ATRISKFOLWUPNTS_TX: NA
    """: "0",
    """
    PNT_NM: Housekeeping - Generation
    QUALIFIER_TXT: Job site hazards, Tripping Hazards
    PNT_ATRISKNOTES_TX: An employee was descending a staircase and when stepping down from the last step she rolled her
    ankle on an extension cord on the floor. She suffered a torn ligament and a broken ankle that resulted in
    persistent pain for more than a year.
    PNT_ATRISKFOLWUPNTS_TX: NA 
    """: "1",
    """
    PNT_NM: Did you recognize additional slip, trip, or fall hazards that had not already been recognized and mitigated? If so, please select or describe these hazards in the At-Risk notes.
    QUALIFIER_TXT: Awareness of environment
    PNT_ATRISKNOTES_TX: [NAME] was working a near by cliff that had about a 20' drop off, crew didn't discuss as a hazard on briefing, i discussed with GF and he told the foreman to make the corrections and place something out there to give crews a visual.
    PNT_ATRISKFOLWUPNTS_TX: NA
    """: "2",
}

#GenerateWeakLabels(classes=classes, shot_examples=multishot, column_label='clsss')
#df_subsample

In [14]:
class GenerateWeakLabels:
    def __init__(
            self,
            classes: Dict[int, str],
            column_label: str,
            shot_examples: Optional[Dict[str, str]] = None,
            df: pd.DataFrame = df_subsample,
            model: str = 'llama3:latest',
            max_workers: int = 16,  # Number of threads/processes
        ):  
        """Generate weak labels for a DataFrame using an LLM."""
        self.classes = classes
        self.column_label = column_label
        self.shot_examples = shot_examples
        self.df = df
        self.model = model

        self.df[column_label] = self._generate_predictions(max_workers)

    def _generate_predictions(self, max_workers: int):
        predictions = []

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_row = {executor.submit(self._process_row, row): row for _, row in self.df.iterrows()}
            
            for future in as_completed(future_to_row):
                row = future_to_row[future]
                try:
                    output = future.result()
                    if not output.isnumeric() or int(output) not in list(classes.keys()):
                        print(output, list(classes.keys()))
                        output=list(classes.keys())[-1]
                    predictions.append(output)
                except Exception as e:
                    print(f"Error processing row {row['PNT_NM']}: {e}")
                    predictions.append(None)  # Handle the error as needed

        return predictions

    def _process_row(self, row):
        prompt_field_notes = "\n".join(f"{key}: {row[key]}" for key in ['PNT_NM', 'QUALIFIER_TXT', 'PNT_ATRISKNOTES_TX', 'PNT_ATRISKFOLWUPNTS_TX'])
        #print(prompt_field_notes)

        output = ollama.chat(
            model=self.model, 
            messages=self._build_shot_prompts(self.classes, prompt_field_notes, self.shot_examples),
            options={'temperature': 0, 'num_predict': 1}
        )

        #print(output['message']['content'])
        return output['message']['content']

    def _build_shot_prompts(self, classes, prompt_field_notes, shot_examples):
        prompt_classes = "\n".join(f"{k}: -> {v}" for k, v in classes.items())

        messages = [{'role': 'system', 'content': SYSTEM_PROMPT_FORMAT.format(classes=prompt_classes)}]

        if shot_examples is None:
            messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=prompt_field_notes)})
            return messages

        for example in shot_examples:
            messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=example)})
            messages.append({'role': 'assistant', 'content': str(shot_examples[example])})

        messages.append({'role': 'user', 'content': USER_PROMPT_FORMAT.format(field_notes=prompt_field_notes)})
        return messages

    def _pretty_print_messages(self, messages):
        for message in messages:
            role = message['role'].capitalize()
            content = message['content']
            print(f"{role}: {content}\n")

In [15]:
GenerateWeakLabels(classes=classes, shot_examples=multishot, column_label='cls')

df_subsample

S [0, 1, 2, 3]


/var/folders/1h/mw4kschx1jjdmd14bjl9nv4r0000gn/T/ipykernel_16061/1480196528.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[column_label] = self._generate_predictions(max_workers)


,DATETIME_DTM,PNT_NM,QUALIFIER_TXT,PNT_ATRISKNOTES_TX,PNT_ATRISKFOLWUPNTS_TX,cls
OBSRVTN_NB,,,,,,
330560,2023-03-15 11:01:00,"Did you recognize additional slip, trip, or fa...",Awareness of environment,[NAME] was working a near by cliff that had ab...,NA,0
466285,2024-05-14 14:20:00,Complete job briefing given,Voltage being worked discussed,Voltage being worked was not documented on job...,NA,3
297337,2022-11-17 08:05:00,Housekeeping in order (Forestry),Cleanliness,Internal truck boxes in very bad organization....,NA,3
440371,2024-02-26 11:30:00,Complete job briefing given,Job briefing conducted and documented,The crew showed up to the job site before the ...,NA,3
324373,2023-02-24 19:29:00,Traffic Control,Other - Traffic Control,Did have a coaching moment while they were set...,NA,0
...,...,...,...,...,...,...
322623,2023-02-20 23:17:00,THA,Reviewed with all parties on-site,The crew had not checked the boxes for the PM ...,NA,3
79958,2021-02-18 13:46:00,Housekeeping in order,"Trash, tools, equipment secured on vehicle",All 4 trucks were left running with gear and e...,NA,3
263543,2022-08-01 09:45:00,Effective Communication,3-Way Communication,Reviewed making sure all crew members are comm...,NA,3


In [16]:
y = np.array(df_subsample['cls'], dtype=np.int32)
(y==0).sum(),(y==1).sum(),(y==2).sum(),(y==3).sum()

(15, 0, 1, 84)